## Homework 2: Decision Tree and Random Forest Implementation on Abalone Dataset

**Objective:**  
In this assignment, we implement a decision tree classifier and a random decision forest entirely from scratch, without using external libraries such as `scikit-learn` for model training.  
We apply our models on the Abalone dataset, which contains physical measurements of abalone and aims to predict the number of rings (an approximation of age).  
The dataset includes both categorical (Sex) and numeric features, making it a suitable candidate for tree-based models.

---

**Improving Accuracy with Label Grouping:**  
In addition to using the original number of rings (ranging from 1 to 29) as classification labels, we introduce a grouped classification scheme to reduce class imbalance and increase model robustness:
- Class 0 (Young): Rings 1–8  
- Class 1 (Middle-aged): Rings 9–10  
- Class 2 (Old): Rings 11 and above

This grouping simplifies the classification task and enables the models to achieve more stable and interpretable results.

---

**Why Accuracy is Relatively Low with Original Labels:**  
- The Abalone dataset is highly overlapping, where different input combinations may yield similar target values.
- Although this is originally a regression problem, it is treated here as a multi-class classification task, increasing complexity.
- Some ring classes are extremely rare, making the data highly imbalanced and prone to overfitting or underfitting.
- Shallow decision trees underfit the data, while unpruned deep trees overfit.
- Random forests reduce variance, but due to noise and label complexity, the overall accuracy may still remain modest.

---

**Performance Evaluation:**  
We evaluate the models using 5-fold cross validation. For each fold:
- Accuracy is computed
- Confusion matrices are printed

This evaluation is done for both:
- Original multi-class labels (1–29 rings)
- Grouped labels (3-category age bins)

---

**Structure of This Notebook:**
1. Data loading and preprocessing (original and grouped labels)
2. Implementation of decision tree construction
3. Prediction function for decision trees
4. Evaluation with k-fold cross validation and confusion matrices
5. Pruned decision tree and evaluation
6. Random forest implementation
7. Random forest evaluation

This notebook helps build a deep understanding of how decision trees and ensemble methods operate on real-world mixed-type data.


### 1. Implementation of Decision Tree Modeling Function

This cell begins by loading the Abalone dataset, encoding the categorical feature (`Sex`) as integers, and separating the data into:
- `X`: the feature matrix
- `y`: the target vector (original ring counts)
- `y_grouped`: an alternative target vector where ring counts are grouped into 3 age-based classes

We also define the `attribute_types` list to indicate the type of each feature (categorical or numeric).

Following preprocessing, we implement the core logic of the decision tree construction:
- `entropy(y)`: computes class impurity for a given label distribution
- `split_data(...)`: splits the dataset based on attribute type and threshold
- `build_dt(...)`: recursively builds a decision tree by selecting the best attribute and threshold based on information gain
- `Node` class: represents each node in the tree and stores the attribute index, split value, child nodes, and predicted label at leaves

In [20]:
import pandas as pd
import numpy as np

# Column name
columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight',
           'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']

# Load data
data = pd.read_csv("abalone.data", names=columns)

# Encode categorical (sex)
data['Sex'] = data['Sex'].astype('category').cat.codes

X = data.drop('Rings', axis=1).values
y = data['Rings'].values

# Alternative path
def convert_labels_to_classes(y):
    y_new = np.zeros_like(y)
    y_new[(y >= 1) & (y <= 8)] = 0  # Young
    y_new[(y >= 9) & (y <= 10)] = 1  # Middle
    y_new[y >= 11] = 2  # Old
    return y_new

y_grouped = convert_labels_to_classes(y)


# Attribute types: 2 = categorical, 1 = numeric
attribute_types = [2] + [1] * 7

# Print first 5 row  
print(data.head())
print("Data Size", X.shape)

from collections import Counter

def entropy(y):
    counts = np.bincount(y)
    probs = counts[np.nonzero(counts)] / len(y)
    return -np.sum(probs * np.log2(probs))

def split_data(X, y, attr, value, is_categorical):
    if is_categorical:
        idx = X[:, attr] == value
    else:
        idx = X[:, attr] <= value
    return X[idx], y[idx], X[~idx], y[~idx]

class Node:
    def __init__(self, attr=None, value=None, left=None, right=None, *, label=None):
        self.attr = attr
        self.value = value
        self.left = left
        self.right = right
        self.label = label

def build_dt(X, y, attribute_types, options=None):
    if len(set(y)) == 1:
        return Node(label=y[0])
    if X.shape[1] == 0 or len(y) == 0:
        return Node(label=Counter(y).most_common(1)[0][0])

    best_gain = -1
    best_attr, best_val = None, None
    best_splits = None

    for i in range(X.shape[1]):
        values = np.unique(X[:, i])
        for val in values:
            Xl, yl, Xr, yr = split_data(X, y, i, val, attribute_types[i] == 2)
            if len(yl) == 0 or len(yr) == 0:
                continue
            gain = entropy(y) - (len(yl)/len(y))*entropy(yl) - (len(yr)/len(y))*entropy(yr)
            if gain > best_gain:
                best_gain = gain
                best_attr = i
                best_val = val
                best_splits = (Xl, yl, Xr, yr)

    if best_gain == -1:
        return Node(label=Counter(y).most_common(1)[0][0])

    left = build_dt(best_splits[0], best_splits[1], attribute_types, options)
    right = build_dt(best_splits[2], best_splits[3], attribute_types, options)
    return Node(attr=best_attr, value=best_val, left=left, right=right)


   Sex  Length  Diameter  Height  Whole weight  Shucked weight  \
0    2   0.455     0.365   0.095        0.5140          0.2245   
1    2   0.350     0.265   0.090        0.2255          0.0995   
2    0   0.530     0.420   0.135        0.6770          0.2565   
3    2   0.440     0.365   0.125        0.5160          0.2155   
4    1   0.330     0.255   0.080        0.2050          0.0895   

   Viscera weight  Shell weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  
Data Size (4177, 8)


### 2. Implementation of Decision Tree Testing Function

This cell implements the `predict_dt` function, which applies a trained decision tree to a set of test samples.

The function relies on a recursive helper method `traverse(node, x)` to classify a single sample by walking down the tree:
- At each decision node, it checks whether the splitting attribute is categorical or numeric.
- Based on the attribute type and the sample's feature value, it chooses the appropriate child (left or right).
- Once a terminal (leaf) node is reached, the corresponding class label is returned.

The outer function applies this traversal to each row in the input matrix `X` and returns a vector of predicted class labels.

In [2]:
def predict_dt(dt, X, options=None):
    def traverse(node, x):
        if node.label is not None:
            return node.label
        val = x[node.attr]
        if attribute_types[node.attr] == 2:
            branch = node.left if val == node.value else node.right
        else:
            branch = node.left if val <= node.value else node.right
        return traverse(branch, x)
    return np.array([traverse(dt, x) for x in X])

### 3. Results of k-Fold Cross Validation

In this cell, we evaluate the decision tree classifier using 5-fold cross validation on both the original labels and the grouped 3-class labels.

The `evaluate_model_versions` function performs the following steps for each fold:
- Splits the data into training and testing sets,
- Trains a decision tree separately on both `y` (original ring counts) and `y_grouped` (age-based classes),
- Predicts the labels on the test set for each version,
- Calculates and stores the accuracy for each fold,
- Computes and prints the mean accuracy across all folds.

This dual evaluation setup allows us to compare the impact of class grouping on classification performance.

In [19]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score

def evaluate_model_versions(X, y_original, y_grouped, attribute_types, builder, predictor, options=None, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accs_orig, accs_grouped = [], []
    confs_orig, confs_grouped = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
        # Orjinal
        dt_orig = builder(X[train_idx], y_original[train_idx], attribute_types, options)
        y_pred_orig = predictor(dt_orig, X[test_idx], options)
        accs_orig.append(accuracy_score(y_original[test_idx], y_pred_orig))
        confs_orig.append(confusion_matrix(y_original[test_idx], y_pred_orig))

        # Group
        dt_grp = builder(X[train_idx], y_grouped[train_idx], attribute_types, options)
        y_pred_grp = predictor(dt_grp, X[test_idx], options)
        accs_grouped.append(accuracy_score(y_grouped[test_idx], y_pred_grp))
        confs_grouped.append(confusion_matrix(y_grouped[test_idx], y_pred_grp))

   
    print("Original Accuracy Scores:", accs_orig)
    print("Mean Accuracy (Original):", np.mean(accs_orig))
    print("\nGrouped Accuracy Scores:", accs_grouped)
    print("Mean Accuracy (Grouped):", np.mean(accs_grouped))

 
    for i, (conf_o, conf_g) in enumerate(zip(confs_orig, confs_grouped), 1):
        print(f"\nFold {i} - Original Confusion Matrix:\n{conf_o}")
        print(f"Fold {i} - Grouped Confusion Matrix:\n{conf_g}")

# Evaluate the model
evaluate_model_versions(X, y, y_grouped, attribute_types, build_dt, predict_dt)

Original Accuracy Scores: [0.19856459330143542, 0.22488038277511962, 0.19640718562874251, 0.19401197604790418, 0.17485029940119762]
Mean Accuracy (Original): 0.1977428874308799

Grouped Accuracy Scores: [0.5885167464114832, 0.5741626794258373, 0.5652694610778443, 0.5796407185628742, 0.5532934131736527]
Mean Accuracy (Grouped): 0.5721766037303384

Fold 1 - Original Confusion Matrix:
[[ 1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  2  3  3  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  6 11  8  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  8 10 13 10  2  1  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  5 17 16 21  8  6  4  2  1  1  1  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  1  4 11 26 30 13  8  2  1  1  1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  6 32 29 26 23  6  8  5  3  0  1  0  1  0  0  0  0  0  0]
 [ 0  0  0  2  6 12 30 33 23  9  8  4  2  2  4  0  2  1  1  0  0  0  0]
 [ 0  0  0  1  4 11 12 20 20  7  6  2  

### 4. Implementation of Decision Tree Function with Depth-Based Pruning

This cell introduces a depth-limited variant of the decision tree builder via the `build_dt_pruned` function.

Key enhancements:
- A `max_depth` parameter is added to restrict the depth of the tree.
- During recursion, if the current depth reaches or exceeds `max_depth`, the algorithm stops further splitting and returns a leaf node with the majority class label in the current subset.
- This approach helps prevent overfitting by controlling model complexity.

All other aspects of the decision tree construction—such as entropy calculation, data splitting, and selection of the best split based on information gain—remain unchanged from the original `build_dt` implementation.

In [17]:
def build_dt_pruned(X, y, attribute_types, max_depth, depth=0):
    if len(set(y)) == 1 or depth >= max_depth:
        return Node(label=Counter(y).most_common(1)[0][0])

    best_gain = -1
    best_attr, best_val = None, None
    best_splits = None

    for i in range(X.shape[1]):
        values = np.unique(X[:, i])
        for val in values:
            Xl, yl, Xr, yr = split_data(X, y, i, val, attribute_types[i] == 2)
            if len(yl) == 0 or len(yr) == 0:
                continue
            gain = entropy(y) - (len(yl)/len(y))*entropy(yl) - (len(yr)/len(y))*entropy(yr)
            if gain > best_gain:
                best_gain = gain
                best_attr = i
                best_val = val
                best_splits = (Xl, yl, Xr, yr)

    if best_gain == -1:
        return Node(label=Counter(y).most_common(1)[0][0])

    left = build_dt_pruned(best_splits[0], best_splits[1], attribute_types, max_depth, depth + 1)
    right = build_dt_pruned(best_splits[2], best_splits[3], attribute_types, max_depth, depth + 1)
    return Node(attr=best_attr, value=best_val, left=left, right=right)


### 5. Results of k-Fold Cross Validation (with Pruning)

In this cell, we evaluate the performance of the depth-pruned decision tree using 5-fold cross validation on both the original and grouped label sets.

The `evaluate_model_versions` function is used in combination with a lambda wrapper around `build_dt_pruned`, where the maximum tree depth is explicitly limited (e.g., `max_depth=5`).

This setup allows us to observe how depth-based pruning influences model generalization by:
- Reducing tree complexity,
- Controlling overfitting on the training data.

The printed results include fold-wise accuracy values and their mean for both labeling strategies, enabling a direct comparison with the unpruned model.

In [18]:
evaluate_model_versions(X, y,y_grouped, attribute_types, lambda x, y, t, o: build_dt_pruned(x, y, t, 5), predict_dt)

Original Accuracy Scores: [0.28827751196172247, 0.27751196172248804, 0.2622754491017964, 0.22994011976047904, 0.2682634730538922]
Mean Accuracy (Original): 0.2652537031200756

Grouped Accuracy Scores: [0.6100478468899522, 0.6255980861244019, 0.6287425149700598, 0.6239520958083832, 0.6095808383233533]
Mean Accuracy (Grouped): 0.61958427642323

Fold 1 - Original Confusion Matrix:
[[ 2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  9  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 12 10  7  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  7 19 13  5  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  2 18 23 24 12  4  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  1 10 48 25 11  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  7 33 46 48  6  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  2 29 29 61 10  0  1  0  0  7  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  6 21 39 19  0  0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  4  7 22

### 6. Implementation of Random Decision Forest (RDF)

This cell implements a basic version of a Random Decision Forest (RDF) by constructing and combining multiple decision trees.

- `build_rdf`: Generates an ensemble of `N` decision trees, where each tree is trained on a different bootstrap sample drawn with replacement from the training set.
- `predict_rdf`: Applies all trees to the test set and performs majority voting across the individual predictions to determine the final class labels.

This ensemble method improves model stability and prediction accuracy by reducing the variance associated with individual decision trees.

In [13]:

def build_rdf(X, y, attribute_types, N, options=None):
    trees = []
    rng = np.random.default_rng(42)
    for _ in range(N):
        indices = rng.choice(len(X), len(X), replace=True)
        X_sample, y_sample = X[indices], y[indices]
        tree = build_dt(X_sample, y_sample, attribute_types, options)
        trees.append(tree)
    return trees


def predict_rdf(rdf, X, options=None):
    predictions = np.array([predict_dt(tree, X) for tree in rdf])
    return np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=predictions)


### 7. Results of k-Fold Cross Validation (Random Decision Forest)

In this cell, we evaluate the Random Decision Forest (RDF) using 5-fold cross validation on both the original and grouped class labels.

The `evaluate_rdf_versions` function performs the following operations for each fold:
- Constructs an ensemble of `N` decision trees using bootstrapped training samples,
- Predicts the test set labels using majority voting across the ensemble,
- Computes and stores the classification accuracy for both the original and grouped target formats.

After all folds are processed, the function prints per-fold accuracy and the mean accuracy, followed by confusion matrices for each fold and label format.  
This evaluation allows for a direct comparison of ensemble performance relative to single decision trees, as well as the impact of label grouping.

In [11]:
def evaluate_rdf_versions(X, y_original, y_grouped, attribute_types, N, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accs_orig, accs_grouped = [], []
    confs_orig, confs_grouped = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
        # RDF Orjinal
        rdf_orig = build_rdf(X[train_idx], y_original[train_idx], attribute_types, N)
        y_pred_orig = predict_rdf(rdf_orig, X[test_idx])
        accs_orig.append(accuracy_score(y_original[test_idx], y_pred_orig))
        confs_orig.append(confusion_matrix(y_original[test_idx], y_pred_orig))

        # RDF Group
        rdf_grp = build_rdf(X[train_idx], y_grouped[train_idx], attribute_types, N)
        y_pred_grp = predict_rdf(rdf_grp, X[test_idx])
        accs_grouped.append(accuracy_score(y_grouped[test_idx], y_pred_grp))
        confs_grouped.append(confusion_matrix(y_grouped[test_idx], y_pred_grp))

    
    print("RDF Original Accuracy Scores:", accs_orig)
    print("Mean Accuracy (Original):", np.mean(accs_orig))
    print("\nRDF Grouped Accuracy Scores:", accs_grouped)
    print("Mean Accuracy (Grouped):", np.mean(accs_grouped))

   
    for i, (conf_o, conf_g) in enumerate(zip(confs_orig, confs_grouped), 1):
        print(f"\nFold {i} - RDF Original Confusion Matrix:\n{conf_o}")
        print(f"Fold {i} - RDF Grouped Confusion Matrix:\n{conf_g}")

evaluate_rdf_versions(X, y, y_grouped, attribute_types, 5)

RDF Original Accuracy Scores: [0.2284688995215311, 0.22727272727272727, 0.2155688622754491, 0.22275449101796407, 0.1844311377245509]
Mean Accuracy (Original): 0.2156992235624445

RDF Grouped Accuracy Scores: [0.6160287081339713, 0.5956937799043063, 0.6227544910179641, 0.6071856287425149, 0.6191616766467066]
Mean Accuracy (Grouped): 0.6121648568890927

Fold 1 - RDF Original Confusion Matrix:
[[ 1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  8  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  6 11 10  3  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  5 12 15 11  2  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3 16 22 22 10  5  3  1  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  3 16 33 19 12  5  4  0  3  0  1  1  0  0  0  0  0  0]
 [ 0  0  0  2 11 21 36 37 18  7  6  2  1  0  1  0  0  0  0  0  0]
 [ 0  0  0  3  6 11 34 32 22  6 10  3  6  2  1  1  1  1  0  0  0]
 [ 0  0  0  0  1  8 18 29 16  8  5  2  1  2  2  0  0  0  1  0  0]
 [ 0  0  0  